In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from glob import glob
import mysql.connector

import time
from bs4 import BeautifulSoup
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import json
import folium
import warnings
warnings.simplefilter(action="ignore",category=FutureWarning)
import numpy as np
import folium
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin

import urllib
from urllib.request import Request
import re

In [3]:
df1 = pd.read_csv('../data/main/company_info.csv')
df2 = pd.read_csv('../data/main/jobkorea_company.csv')

merged = pd.concat([df1,df2])


merged.to_csv('../data/main/merged_company.csv', index=False)

In [6]:
df = pd.read_csv('../data/main/merged_company.csv')

df['기업형태'] = df['기업형태'].replace({
    '대기업': 1,
    '중견기업': 2,
    '중소기업': 3,
    '스타트업': 4,
    '기타': 5,
    '벤처기업': 6,
    '강소기업': 7,
    '공기업': 8,
    '코스닥상장기업': 9,
    '코스피상장기업': 10

})


df['기업형태'] = df['기업형태'].fillna(11)

print(df)

df.to_csv('../data/main/merged_company_ver2.csv', index=False)

                       기업명            설립일  기업형태    사원수     매출액     최저연봉  \
0                      클로봇  2017년 05월 08일   4.0  141 명  134.4억  3,462만원   
1                     로보티즈  1999년 03월 25일   3.0  157 명     NaN  3,130만원   
2                   마루에이치알  2006년 07월 03일   3.0  270 명  203.5억  2,450만원   
3                     휴림로봇  1999년 11월 29일  11.0   63 명     NaN  2,824만원   
4                      세스텍  1999년 09월 27일   3.0  106 명  341.1억  3,456만원   
..                     ...            ...   ...    ...     ...      ...   
548                   유진로봇            NaN  11.0    NaN     NaN      NaN   
549  3D센서 기반의 머신비전 연구 전문기업            NaN  11.0    NaN     NaN      NaN   
550                    대기업            NaN  11.0    NaN     NaN      NaN   
551                 대동에크테크            NaN  11.0    NaN     NaN      NaN   
552               잡뉴스솔로몬서치            NaN  11.0    NaN     NaN      NaN   

         평균연봉     최고연봉                                             주소  
0    6,605 만원  7,915만원     

In [25]:
df = pd.read_csv('../data/main/merged_company_ver2.csv')

# '만원'을 제거하고 숫자로 변환
df['최저연봉'] = df['최저연봉'].fillna(0)
df['평균연봉'] = df['평균연봉'].fillna(0)
df['최고연봉'] = df['최고연봉'].fillna(0)
df['사원수'] = df['사원수'].fillna(0)
df['매출액'] = df['매출액'].fillna(0)
df['최저연봉'] = df['최저연봉'].replace({'만원': '', ',': ''}, regex=True).astype(int)
df['평균연봉'] = df['평균연봉'].replace({'만원': '', ',': ''}, regex=True).astype(int)
df['최고연봉'] = df['최고연봉'].replace({'만원': '', ',': ''}, regex=True).astype(int)
df['사원수'] = df['사원수'].replace({'명': '', ' ': ''}, regex=True).astype(float)

df.to_csv('../data/main/merged_company_ver3.csv', index=False)

In [26]:
def add_suffix_to_address(address):
    if pd.isna(address): 
        return address 
    if "경기" in address:
        return address.replace("경기", "경기도")
    elif "서울" in address:
        return address.replace("서울", "서울특별시")
    elif "인천" in address:
        return address.replace("인천", "인천광역시")
    elif "충남" in address:
        return address.replace("충남", "충청남도")
    elif "부산" in address:
        return address.replace("부산", "부산광역시")
    elif "대전" in address:
        return address.replace("대전", "대전광역시")
    elif "대구" in address:
        return address.replace("대구", "대구광역시")
    elif "울산" in address:
        return address.replace("울산", "울산광역시")
    elif "광주" in address:
        return address.replace("광주", "광주광역시")
    elif "충북" in address:
        return address.replace("충북", "충청북도")
    elif "경남" in address:
        return address.replace("경남", "경상남도")
    elif "경북" in address:
        return address.replace("경북", "경상북도")
    elif "전남" in address:
        return address.replace("전남", "전라남도")
    elif "전북" in address:
        return address.replace("전북", "전라북도")
    elif "충남" in address:
        return address.replace("충남", "충청남도")
    elif "제주" in address:
        return address.replace("제주", "제주도")
    elif "강원" in address:
        return address.replace("강원", "강원도")
    
    return address


df = pd.read_csv('../data/main/merged_company_ver3.csv')

df['주소'] = df['주소'].apply(add_suffix_to_address)

df.to_csv('../data/main/merged_company_ver4.csv', index=False)

In [27]:
def extract_and_split_city_district(address):
    if not isinstance(address, str): 
        return None, None
    
    match = re.search(r'([가-힣]+(?:도|시|광역시|특별시))\s*([가-힣]+(?:구|시|군))?', address)
    if match:
        city = match.group(1)  
        district = match.group(2)  
        return city, district
    return None, None 


df = pd.read_csv('../data/main/merged_company_ver4.csv')


df[['주소', '구']] = df['주소'].apply(lambda x: pd.Series(extract_and_split_city_district(x)))


df.to_csv('../data/main/merged_company_ver5.csv', index=False)

In [28]:
city_mapping = {
    '경기도': 1, '서울특별시': 2, '충청남도': 3, '전라북도': 4, '대전광역시': 5,
    '부산광역시': 6, '인천광역시': 7, '광주광역시': 8, '경상북도': 9, '대구광역시': 10,
    '울산광역시': 11, '경상남도': 12, '전라남도': 13, '강원도': 14, '충청북도': 15,
    '제주도': 16
}
df = pd.read_csv('../data/main/merged_company_ver5.csv')

df['주소'] = df['주소'].map(city_mapping)

df['주소'] = df['주소'].fillna(0)
df.to_csv('../data/main/merged_company_ver6.csv', index=False)

In [29]:
gu_names = [
    '성남시', '강서구', '영등포구', '천안시', '아산시', '마포구', '서초구', '전주시', '강남구', '유성구', '동작구', '수원시', 
    '안양시', '성동구', '미추홀구', '관악구', '북구', '경산시', '사상구', '금천구', '서구', '과천시', '남구', '남동구', '광산구', 
    '부평구', '화성시', '중구', '창원시', '부천시', '동구', '나주시', '파주시', '오산시', '칠곡군', '의왕시', '연수구', '원주시', 
    '용인시', '김해시', '시흥시', '평택시', '거제시', '경주시', '양천구', '동래구', '진주시', '여수시', '남양주시', '김포시', 
    '기장군', '군산시', '송파구', '연제구', '장성군', '안산시', '달서구', '광진구', '포항시', '해운대구', '춘천시', '청주시', 
    '성북구', '서귀포시', '고흥군', '제천시', '노원구', '횡성군', '금정구', '하남시', '익산시', '영동군', '달성군', '수성구', 
    '대덕구', '구로구', '종로구', '중랑구', '가평군', '고양시', '양산시', '광주시', '옥천군', '구리시', '공주시', '광양시', 
    '목포시', '안성시', '동해시', '서대문구', '부산광역시진구', '계양구', '군포시', '광명시', '충주시', "정보없음", '구미시' 
]

# 구 이름에 숫자 매핑
gu_mapping = {name: idx + 1 for idx, name in enumerate(gu_names)}

# 예제 DataFrame
df = pd.read_csv('../data/main/merged_company_ver6.csv')

# 구 이름을 숫자로 대체
df['구'] = df['구'].map(gu_mapping)
df['구'] = df['구'].fillna(0)

df.to_csv('../data/main/merged_company_ver7.csv', index=False)

In [30]:
df = pd.read_csv('../data/main/merged_company_ver7.csv')

df["주소"] = df["주소"].replace(0,17)
df["구"] = df["구"]. replace(0,96)

df.to_csv('../data/main/merged_company_ver8.csv', index=False)

In [31]:
df = pd.read_csv('../data/main/merged_company_ver8.csv')

df = df.replace(r'^\s*$', None, regex=True)

df.to_csv('../data/main/merged_company_ver9.csv', index=False)

In [32]:
def process_foundation_year(date):
    if pd.isna(date):  
        return "정보없음"  
    elif isinstance(date, str):
        if "-" in date:  
            return date.split('-')[0] + "년"  
    return "정보없음"



df = pd.read_csv('../data/main/merged_company_ver9.csv')
df["설립일"] = df["설립일"].str.replace(' ','')
df["설립일"] = df["설립일"].str.replace('년', '-').str.replace('월', '-').str.replace('일', '')


df["설립일"] = df["설립일"].apply(process_foundation_year)

# df["설립일"].head()
df.to_csv('../data/main/merged_company_final.csv', index=False)

In [ ]:
conn = mysql.connector.connect(
host = "database-1.c9ks828mszs7.ap-northeast-2.rds.amazonaws.com",
port = 3306,
user = "root",
password = "asd20187304",
database = "roschool"
)


In [ ]:
cursor = conn.cursor(buffered=True)
sql = "INSERT INTO company_info (sido_id, gungu_id, comp_type_id, comp_name, foundation, employees,income, low_salary, avg_salary, high_salary) VALUES (%s, %s, %s, %s,%s, %s, %s, %s, %s, %s)"
df = pd.read_csv('../data/saramin_data_append_ver9.csv')

for i, row in df.iterrows():
    cursor.execute(sql, (row['주소'], row['구'], row['기업형태'], row['기업명'],row['설립일'], row['사원수'], row['매출액'], row['최저연봉'], row['평균연봉'], row['최고연봉']))
    print(row)
    conn.commit()


In [ ]:
conn.close()